In [29]:
import pandas as pd
import math

In [30]:
# 读取第一个Excel文件
normal = pd.read_excel('Normal.xlsx')

# 读取第二个Excel文件
faulty= pd.read_excel('Faulty.xlsx')

print(normal.shape)
print(faulty.shape)


(4251, 252)
(4251, 68)


In [31]:
normal.head(5)

,0,1,3,6,0.1,2,3.1,4,6.1,7,...,8.28,0.23,1.29,2.25,3.32,4.27,5.25,6.29,7.24,8.29
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [32]:
# 获取每列的行数
column_rows_n = [len(normal[column].dropna()) for column in normal.columns]
print(column_rows_n)

column_rows_f = [len(faulty[column].dropna()) for column in faulty.columns]
print(column_rows_f)

# 确定Normal数据集最小行数
min_rows_n = min(column_rows_n)

print("Normal数据集最小行数：", min_rows_n)

# 确定Faulty数据集最小行数
min_rows_f = min(column_rows_f)

print("Faulty数据集最小行数：", min_rows_f)

min_rows = min(min_rows_n, min_rows_f)
print("全部数据的最小行数：", min_rows)

#我们认为可以使用最小行数作为SPP层深的划分
k=math.floor(math.log2(min_rows))
print("SPP层深：",k)

[2919, 1674, 1412, 1495, 2821, 2064, 729, 3312, 1426, 1180, 1906, 4036, 1375, 2889, 1904, 3036, 1560, 2198, 2288, 1996, 1447, 3075, 3618, 1951, 1891, 2990, 3509, 1352, 3261, 3216, 1672, 1824, 1758, 1549, 2934, 1525, 1147, 1278, 1948, 1666, 3282, 3324, 1409, 1337, 821, 1830, 1735, 3202, 1374, 1300, 1646, 2919, 1674, 1412, 1495, 2821, 2064, 729, 3312, 3412, 1426, 1180, 1906, 1375, 2889, 1904, 2036, 4251, 3394, 1560, 2288, 1996, 3999, 1447, 3075, 1951, 2990, 1886, 3509, 1352, 3216, 1672, 1824, 1758, 1549, 2820, 2934, 1525, 1278, 3850, 1666, 3324, 1409, 1337, 821, 2567, 1830, 3135, 1735, 2941, 1374, 1300, 1646, 2919, 1674, 3814, 1412, 3029, 2864, 1495, 1093, 2340, 2821, 1721, 2064, 729, 3312, 3412, 1426, 1180, 1380, 2590, 1906, 4036, 1375, 2889, 3081, 1390, 1271, 1904, 2036, 4251, 1561, 3036, 1560, 1561, 2198, 2288, 1996, 3999, 1447, 3075, 3618, 1951, 1891, 2990, 3537, 1886, 3509, 1352, 3261, 3216, 1252, 1672, 1824, 1758, 3515, 1549, 2820, 2934, 1525, 1147, 1278, 2521, 1948, 3850, 1666, 32

### 下面开始进行SPP神经网络的构建
#### 暂时使用k=5进行SPP池化

In [33]:
import torch

def split(x):
    assert x.dim()==2
    num_rows = x.shape[0]
    split_indices = torch.arange(1, num_rows, 2)
    x_odd = x[split_indices, :]
    x_even = x[split_indices-1, :]
    return x_odd, x_even

In [34]:
import torch
import torch.nn as nn

class ImprovedSPPLayer(nn.Module):
    def __init__(self, num_levels):
        super(ImprovedSPPLayer, self).__init__()
        self.num_levels = num_levels  # 划分的总层数
        self.num_spp_outputs = 2 ** self.num_levels - 1  # 最终SPP输出的总数量

    def forward(self, x):
        spp_outputs = []
        spp_split = []

        # 第0层：统计每列含1的个数
        ones_count = torch.sum(torch.nan_to_num(x, nan=0), dim=0)
        spp_outputs.append(ones_count.unsqueeze(dim=1))
        spp_split.append(x)

        # 第1到k-1层，对于行分奇偶，然后统计每列含1的个数
        for level in range(1, self.num_levels):
            for i in range(2**(level-1), 2**level):
                x_odd, x_even = split(spp_split[i-1])
                ones_count_odd = torch.sum(torch.nan_to_num(x_odd, nan=0), dim=0)
                ones_count_even = torch.sum(torch.nan_to_num(x_even, nan=0), dim=0)
                spp_outputs.append(ones_count_odd.unsqueeze(dim=1))
                spp_outputs.append(ones_count_even.unsqueeze(dim=1))
                spp_split.append(x_odd)
                spp_split.append(x_even)
        
        # concat 2^k 个数据
        spp_output = torch.cat(spp_outputs, dim=1)
        return spp_output



In [38]:
normal_tensor = torch.from_numpy(normal.values)
faulty_tensor = torch.from_numpy(faulty.values)

improved_spp = ImprovedSPPLayer(num_levels=5)  # 定义改进的SPP层，指定层数
normal_output = improved_spp(normal_tensor)  # 改进的SPP池化后的输出
print(normal_output.shape)  # 打印输出
faulty_output = improved_spp(faulty_tensor)
print(faulty_output.shape)

torch.Size([252, 31])
torch.Size([68, 31])


In [22]:
normal_tensor.shape

torch.Size([4251, 252])

In [14]:
normal.shape[1] 

252